### 한빛아카데미 - 새로나온 책
- https://www.hanbit.co.kr/academy/books/new_book_list.html?page=1 (메인)
- https://www.hanbit.co.kr/academy/books/book_view.html?p_code=B7192437175 (서브)
- location='/academy/books/book_view.html?p_code=B7192437175' (알 수 있는 정보)
- 제목, 저자, 출간일, 쪽수, 가격, 역자

In [1]:
import requests
import pandas as pd
from tqdm import tqdm
from bs4 import BeautifulSoup

##### 1. 데이터 가져오기

In [2]:
base_url = 'https://www.hanbit.co.kr'
main_url = '/academy/books/new_book_list.html'
url = f'{base_url}{main_url}?page=1'
result = requests.get(url)
soup = BeautifulSoup(result.text)

##### 2. 찾으려고 하는 데이터 찾기

In [3]:
lis = soup.select('li.sub_book_list')
len(lis)

20

##### 3. 하나의 데이터로 원하는 정보 추출

In [4]:
li = lis[0]
href = li.select_one('.pop_quick_bg')['onclick']
href

"location='/academy/books/book_view.html?p_code=B7192437175'"

In [5]:
href = href[10:-1]
href

'/academy/books/book_view.html?p_code=B7192437175'

- sub page에서 정보 추출

In [6]:
sub_url = f'{base_url}{href}'
res = requests.get(sub_url)
book_soup = BeautifulSoup(res.text)

In [7]:
info = book_soup.select_one('.store_product_info_box')
title = info.find('h3').text.strip()
title

'처음 배우는 매트랩'

In [8]:
# 저자
book_lis = info.select('.info_list > li')
author = ', '.join(book_lis[0].find('span').get_text().strip().split(' , '))
author

'방성완'

In [9]:
# 역자
if len(book_lis) == 5:
    translator = '-'
else:
    translator = ', '.join(book_lis[1].find('span').get_text().strip().split(' , '))
translator

'-'

In [10]:
# 출간일
date = book_lis[-4].find('span').get_text().strip()
date

'2023-04-10'

In [11]:
# 페이지
page = int(book_lis[-3].find('span').get_text().strip()[:-2])
page

416

In [12]:
# 가격
payments = book_soup.select_one('.payment_box.curr').find_all('p')
sales = payments[1].select_one('.pbr').find('strong').get_text().strip()
sales

'30,000'

##### 4. 한 페이지에 있는 데이터

In [13]:
lines = []
for li in lis:
    href = li.select_one('.pop_quick_bg')['onclick'][10:-1]
    res = requests.get(base_url + href)
    book_soup = BeautifulSoup(res.text)
    info = book_soup.select_one('.store_product_info_box')
    title = info.find('h3').text.strip()
    book_lis = info.select('.info_list > li')
    author = ', '.join(book_lis[0].find('span').get_text().strip().split(' , '))
    if len(book_lis) == 5:
        translator = '-'
    else:
        translator = ', '.join(book_lis[1].find('span').get_text().strip().split(' , '))
    date = book_lis[-4].find('span').get_text().strip()
    page = int(book_lis[-3].find('span').get_text().strip()[:-2])
    payments = book_soup.select_one('.payment_box.curr').find_all('p')
    sales = payments[1].select_one('.pbr').find('strong').get_text().strip()
    lines.append({
        '제목':title, '저자':author, '역자':translator,
        '출간일':date, '페이지수':page, '판매가':sales
    })

In [14]:
df1 = pd.DataFrame(lines)
df1

,제목,저자,역자,출간일,페이지수,판매가
0,처음 배우는 매트랩,방성완,-,2023-04-10,416,"30,000"
1,SAS와 메타분석을 활용한 한번에 적용하는 의학보건통계,히든그레이스 데이터분석팀,-,2023-03-09,516,"34,000"
2,정보교과교육론(3판),"최현종, 전용주",-,2023-03-01,488,"30,000"
3,"IT CookBook, 기초전자실험 with PSpice(3판)",홍순관,-,2023-02-28,308,"22,000"
4,경제수학 강의(3판),김성현,-,2023-02-28,464,"23,000"
5,글로벌 사회와 리더십(2판),안광현,-,2023-02-28,416,"28,000"
6,행동 재무론,고광수,-,2023-02-28,596,"32,000"
7,"IT CookBook, 알기 쉽게 해설한 파이썬 with 컴퓨팅 사고력","김충석, 김지연",-,2023-02-24,328,"24,000"
8,MATLAB으로 배우는 공학 수치해석(3판),방성완,2023-02-20,360 쪽,97911566463,"28,000"
9,Cengel의 알기 쉬운 열역학(6판),"Yunus A. Çengel, John M. Cimbala, Afshin J. Gh...","김택영, 한진호, 김덕종, 김우태, 이무연, 전문수",2023-02-17,528,"34,000"


5. 오류해결

In [16]:
# 에러가 발생한 요소
li = lis[8] # 위에 보면 8번째 에러확인됨
href = li.select_one('.pop_quick_bg')['onclick'][10:-1]
res = requests.get(base_url + href)
book_soup = BeautifulSoup(res.text)

In [17]:
book_lis = info.select('.info_list > li')
for li in book_lis:
    print(li.get_text())

저자 : Peter G. Northouse 
번역 : 김남현 
출간 : 2023-01-02
페이지 : 660 쪽
ISBN : 9791156646419
물류코드 :4641


In [18]:
# 딕셔너리 만들기 프레임 / 키값을 주면 밸류값이 나오니까
book_dict = {}
for li in book_lis:
    item = li.get_text().split(':')
    book_dict[item[0].strip()] = item[1].strip()
book_dict

{'저자': 'Peter G. Northouse',
 '번역': '김남현',
 '출간': '2023-01-02',
 '페이지': '660 쪽',
 'ISBN': '9791156646419',
 '물류코드': '4641'}

In [21]:
# author = ', '.join(book_lis[0].find('span').get_text().strip().split(' , '))
# 딕셔너리를 통해서 author 바뀜
author = ', '.join(book_dict['저자'].split(' , '))
author

'Peter G. Northouse'

기존 translator
    if len(book_lis) == 5:
        translator = '-'
    else:
        translator = ', '.join(book_lis[1].find('span').get_text().strip().split(' , '))

In [22]:
# translator
if '번역' in book_dict.keys(): # 번역이라는 단어가 북딕트의 키에속하는지
    translator = ', '.join(book_dict['번역'].split(' , '))
else:
    translator = '-'
translator

'김남현'

In [24]:
# translator 한줄로
translator = ', '.join(book_dict['번역'].split(' , ')) if '번역' in book_dict.keys() else '-'
translator

'김남현'

In [ ]:
# 출간일
# date = book_lis[-4].find('span').get_text().strip()
data = book_dict['출간'] # strip위에서 해서 더이상 안해도됨다고.....
date

In [25]:
# 페이지
# page = int(book_lis[-3].find('span').get_text().strip()[:-2])
page = int(book_dict['페이지'][:-2])
page

660

In [26]:
lines = []
for li in lis:
    href = li.select_one('.pop_quick_bg')['onclick'][10:-1]
    res = requests.get(base_url + href)
    book_soup = BeautifulSoup(res.text)
    info = book_soup.select_one('.store_product_info_box')
    title = info.find('h3').text.strip()
    book_lis = info.select('.info_list > li')
    book_dict = {}
    for li in book_lis:
        item = li.get_text().split(':')
        book_dict[item[0].strip()] = item[1].strip()
    author = ', '.join(book_dict['저자'].split(' , '))
    translator = ', '.join(book_dict['번역'].split(' , ')) if '번역' in book_dict.keys() else '-'
    data = book_dict['출간']
    page = int(book_dict['페이지'][:-2])
    payments = book_soup.select_one('.payment_box.curr').find_all('p')
    sales = payments[1].select_one('.pbr').find('strong').get_text().strip()
    lines.append({
        '제목':title, '저자':author, '역자':translator,
        '출간일':date, '페이지수':page, '판매가':sales
    })

In [27]:
df2  =pd.DataFrame(lines)
df2

,제목,저자,역자,출간일,페이지수,판매가
0,처음 배우는 매트랩,방성완,-,2023-01-02,416,"30,000"
1,SAS와 메타분석을 활용한 한번에 적용하는 의학보건통계,히든그레이스 데이터분석팀,-,2023-01-02,516,"34,000"
2,정보교과교육론(3판),"최현종, 전용주",-,2023-01-02,488,"30,000"
3,"IT CookBook, 기초전자실험 with PSpice(3판)",홍순관,-,2023-01-02,308,"22,000"
4,경제수학 강의(3판),김성현,-,2023-01-02,464,"23,000"
5,글로벌 사회와 리더십(2판),안광현,-,2023-01-02,416,"28,000"
6,행동 재무론,고광수,-,2023-01-02,596,"32,000"
7,"IT CookBook, 알기 쉽게 해설한 파이썬 with 컴퓨팅 사고력","김충석, 김지연",-,2023-01-02,328,"24,000"
8,MATLAB으로 배우는 공학 수치해석(3판),방성완,-,2023-01-02,360,"28,000"
9,Cengel의 알기 쉬운 열역학(6판),"Yunus A. Çengel, John M. Cimbala, Afshin J. Gh...","김택영, 한진호, 김덕종, 김우태, 이무연, 전문수",2023-01-02,528,"34,000"


##### 6. 모든 페이지에 대해 적용

In [28]:
lines = []
for page in tqdm(range(1, 24)):   # 총 페이지가 23 페이지
    url = f'{base_url}{main_url}?page={page}'
    result = requests.get(url)
    soup = BeautifulSoup(result.text)
    lis = soup.select('li.sub_book_list')
    for li in lis:
        href = li.select_one('.pop_quick_bg')['onclick'][10:-1]
        res = requests.get(base_url + href)
        book_soup = BeautifulSoup(res.text)
        info = book_soup.select_one('.store_product_info_box')
        title = info.find('h3').text.strip()
        book_lis = info.select('.info_list > li')
        book_dict = {}
        for li in book_lis:
            item = li.get_text().split(':')
            book_dict[item[0].strip()] = item[1].strip()
        author = ', '.join(book_dict['저자'].split(' , '))
        translator = ', '.join(book_dict['번역'].split(' , ')) if '번역' in book_dict.keys() else '-'
        data = book_dict['출간']
        page = int(book_dict['페이지'][:-2])
        payments = book_soup.select_one('.payment_box.curr').find_all('p')
        sales = payments[1].select_one('.pbr').find('strong').get_text().strip()
        lines.append({
            '제목':title, '저자':author, '역자':translator,
            '출간일':date, '페이지수':page, '판매가':sales
        })

100%|██████████| 23/23 [03:39<00:00,  9.55s/it]


##### 7. 데이터프레임 만들고 저장

In [30]:
df3  =pd.DataFrame(lines)
df3.tail()

,제목,저자,역자,출간일,페이지수,판매가
448,문제해결력을 키우는 이공계 글쓰기 : 발표와 토론까지,한성우,-,2023-01-02,396,"18,000"
449,Essential Calculus,James Stewart,-,2023-01-02,960,"36,000"
450,의사결정회계,정재권,-,2023-01-02,506,"28,000"
451,"IT CookBook, 디지털 통신 시스템 : Digital Communicatio...","정호영, 임승각",-,2023-01-02,576,"25,000"
452,인간관계론,"유기현, 송병선, 권용만",-,2023-01-02,415,"25,000"


In [31]:
df3.to_csv('data/20230425_한빛_새로나온책.csv', index=False)